In [ ]:
# define function defining each paper as a GEO or SRA paper...
# ...based on which is mentioned more, or 'Both' if score is equal
def getRepo(paper_IDs):
    try:
        r = statistics.mode(paper_IDs)
    except:
        r = 'Both'
    return r

In [ ]:
# Look at each PAPER
# Across GEO and SRA, what percentage of papers reuse data? (stacked bar chart)

# Take a slice of allFactors with the data we want
byPaper = allFactors[['pmc_ID', 'repository', 'total_use_count']]
byPaper = byPaper.dropna(subset = ['repository'])
byPaper = byPaper.groupby(['pmc_ID'], as_index = False).agg({
    'repository': lambda x: getRepo(x), 'total_use_count': 'mean'})
byPaper.columns = ['pmc_ID', 'repository', 'avg_use']

# divide into GEO, SRA, Both
byPaperGEO = byPaper.loc[byPaper['repository'] == 'GEO']
byPaperSRA = byPaper.loc[byPaper['repository'] == 'SRA']
byPaperBOTH = byPaper.loc[byPaper['repository'] == 'Both']

byPaper

In [ ]:
tc = Counter(byPaper['avg_use'])
totalCounts = pd.DataFrame.from_dict(tc, orient = 'index').reset_index()

gc = Counter(byPaperGEO['avg_use'])
geoCounts = pd.DataFrame.from_dict(gc, orient = 'index').reset_index()

sc = Counter(byPaperSRA['avg_use'])
sraCounts = pd.DataFrame.from_dict(sc, orient = 'index').reset_index()

bc = Counter(byPaperBOTH['avg_use'])
bothCounts = pd.DataFrame.from_dict(bc, orient = 'index').reset_index()

for df in [totalCounts, geoCounts, sraCounts, bothCounts]:
    df.columns = ['avg_use', 'num_papers']
    df = df.sort_values(by = 'times_used').reset_index().drop(labels = ['index'], axis = 1)
    # no_reuse is number of papers only mentioning datasets used once
    no_reuse = df['num_papers'][0]
    # tot_papers includes all papers, including those that mentioned at least one reused dataset
    tot_papers = df['num_papers'].sum()
    pct_reuse_papers = 1 - no_reuse/tot_papers
    print(pct_reuse_papers)